In [1]:
import pandas as pd

#### 1. Countries by share of population over threshold, 2022 reanalysis

In [2]:
def stacked_bar_data(alphabetical=False):
    
    data = pd.read_excel("../output/excel/CAMS-Europe-Renalaysis-Countries-Yearly-2018-2022.xlsx")
    
    corresp = {
        'FR': 'France',
        'HR': 'Croatia',
        'HU': 'Hungary',
        'AL': 'Albania',
        'AT': 'Austria',
        'BE': 'Belgium',
        'BG': 'Bulgaria',
        'CH': 'Switzerland',
        'CY': 'Cyprus',
        'CZ': 'Czech Republic',
        'DE': 'Germany',
        'DK': 'Denmark',
        'EE': 'Estonia',
        'EL': 'Greece',
        'IE': 'Ireland',
        'ES': 'Spain',
        'FI': 'Finland',
        'IS': 'Iceland',
        'PT': 'Portugal',
        'RO': 'Romania',
        'RS': 'Serbia',
        'SE': 'Sweden',
        'SI': 'Slovenia',
        'SK': 'Slovakia',
        'TR': 'Turkey',
        'UK': 'United Kingdom',
        'NO': 'Norway',
        'IT': 'Italy',
        'LI': 'Liechtenstein',
        'LT': 'Lithuania',
        'LU': 'Luxembourg',
        'LV': 'Latvia',
        'ME': 'Montenegro',
        'MK': 'North Macedonia',
        'MT': 'Malta',
        'NL': 'Netherlands',
        'PL': 'Poland',
        'UKR': 'Ukraine',
        'XKO': 'Kosovo', # ISO code XK
        'BIH': 'Bosnia and Herzegovina',
        'MDA': 'Moldova'
    }

    data = data[data.Year==2022][['Country code', 
                         '0–5µg/m³ - percentage',
                         '5–10µg/m³ - percentage',
                         '10–15µg/m³ - percentage',
                         '15–20µg/m³ - percentage',
                         '20–25µg/m³ - percentage',
                         '25+ µg/m³ - percentage']]
    
    # Column name
    data['Country name'] = data['Country code'].map(corresp)
    
    # Convert numerical columns to percentage
    for col in data.select_dtypes(include=['float64', 'int64']):
        data[col] = (data[col] * 100)
        
    # Calculate the new buckets
    data['0-5 µg/m³'] = data['0–5µg/m³ - percentage']
    data['5-10 µg/m³'] = data['5–10µg/m³ - percentage']
    data['10-15 µg/m³'] = data['10–15µg/m³ - percentage']
    data['15-20 µg/m³'] = data['15–20µg/m³ - percentage'] 
    data['20+ µg/m³'] = data['20–25µg/m³ - percentage'] + data['25+ µg/m³ - percentage']

    # Select the required columns
    data = data[['Country code', 'Country name', '0-5 µg/m³', '5-10 µg/m³', '10-15 µg/m³', '15-20 µg/m³', '20+ µg/m³']]
    
    if alphabetical:
        data = data.sort_values(by='Country name')
    else:
        # Sort better to worse
        data = data.sort_values(by=list(reversed(['0-5 µg/m³', '5-10 µg/m³', '10-15 µg/m³', '15-20 µg/m³', '20+ µg/m³'])), 
                               ascending=True)

    
    
    # Label names
    data = data.rename(columns={
        "0-5 µg/m³": "Under WHO limits",
        "5-10 µg/m³": "Up to 2x the limits",
        "10-15 µg/m³": "Up to 3x",
        "15-20 µg/m³": "Up to 4x",
        "20+ µg/m³": "Over 4x"

    })
    
    # Saves
    data.to_csv("../datawrapper/stacked-bars.csv", index=False)

In [3]:
stacked_bar_data(alphabetical=False)

FileNotFoundError: [Errno 2] No such file or directory: '../output/excel/CAMS-Europe-Renalaysis-Countries-Yearly-2018-2022.xlsx'

#### 2. Pollution stripes

In [ ]:
def pollution_stripes_data(src, out):
    
    # Reads data
    df = pd.read_excel(src)
    
    # 2022 data
    df = df[df.Day >= "2022-01-01"] 

    
    # ADD THE NUTS3 CODES TO DISPLAY ON THE CHART HERE.
    # The abbreviation between the colons is the ISO2 code
    # used by Datawrapper to display little flags. The second
    # list element is the name to be displayed
    corresp = {
        
         'ITC4C': [':it:', 'Milan'],
         'ITH31': [':it:', 'Verona'],
         'ITH36': [':it:', 'Padova'],
         'PL213': [':pl:', 'Kraków'],
         'PL911': [':pl:', 'Warsaw'],
         'PL22A': [':pl:', 'Katovice'],
         'EL303': [':gr:', 'Athens'],
         'ES300': [':es:', 'Madrid'],
         'PT170': [':pt:', 'Lisbon'],
         'DE300': [':de:', 'Berlin'],
         'FR101': [':fr:', 'Paris'],
         'NL329': [':nl:', 'Amsterdam'],
         'BE100': [':be:', 'Brussels'],
         'RO321': [':ro:', 'Bucaresti'],
         'RS110': [':rs:', 'Belgrade'],
         'HR050': [':hr:', 'Zagreb'],
         'HU110': [':hu:', 'Budapest'],
         'NO081': [':no:', 'Oslo'],
         'SE110': [':se:', 'Stockholm'],
         'DK011': [':dk:', 'Copenhagen'],
         'IS001': [':is:', 'Reykjavík'],

    }

    # Keeps only the data in the dictionary above
    df = df[df['NUTS ID'].isin(corresp.keys())]
    
    # Creates columns with real names, code for datawrapper flags and a combination of both
    df['code'] = df['NUTS ID'].apply(lambda code: corresp[code][0])
    df['city'] = df['NUTS ID'].apply(lambda code: corresp[code][1])
    df['display name'] = df['NUTS ID'].apply(lambda code: corresp[code][0] + " " + corresp[code][1])
    
    # Keeps only columns of interest – note that the columns Day represents the first date on the time agg unit
    # If dealing with weekly data, for instance, the first day of that week will be shown
    df = df[['Day', 'code', 'city', 'display name', 'Daily PM 2.5 average (µg/m³)']]

    # Alphabetical sort
    df = df.sort_values(by='display name', ascending=False)    
    
    # Putting the time observations as columns
    df = df.pivot(index=['city', 'code', 'display name'], columns='Day').reset_index().droplevel(level=0, axis=1)
        
    # This weird customization is recquired by Datawrapper
    df = df.rename(columns = { column: "" for column in 
                               df.columns })
    
    # Saves CSV
    df.to_csv(out, index=False)

In [ ]:
pollution_stripes_data(src="../output/excel/CAMS-Europe-Renalysis-Weekly-2018-2022.xlsx",
                       out="./datawrapper-data/stripes-weekly-2022.csv")

In [ ]:
pollution_stripes_data(src="../output/excel/CAMS-Europe-Forecast-Weekly-2023.xlsx",
                       out="./datawrapper-data/stripes-weekly-2023.csv")

In [ ]:
# pollution_stripes_data(src="../output/excel/CAMS-Europe-Forecast-Daily-2023.xlsx",
#                        out="./datawrapper-data/stripes-daily-2023.csv")

#### 3. Sparklines

In [ ]:
def sparkline_data():
    
    # Needs only 2022 data for this
    df = pd.read_excel("../output/excel/CAMS-Europe-Renalysis-Yearly-2018-2022.xlsx").fillna("")
    
    # ADD THE NUTS3 CODES TO DISPLAY ON THE CHART HERE.
    # The abbreviation between the colons is the ISO2 code
    # used by Datawrapper to display little flags. The second
    # list element is the name to be displayed
    corresp = {
         'ITC4C': [':it:', 'Milan'],
         'ITH31': [':it:', 'Verona'],
         'ITH36': [':it:', 'Padova'],
         'ITC4A': [':it:', 'Cremona'],
         'ITH32': [':it:', 'Vicenza'],
         'ITH35': [':it:', 'Venice'],
        
         'PL213': [':pl:', 'Kraków'],
         'PL911': [':pl:', 'Warsaw'],
         'PL22A': [':pl:', 'Katovice'],
         'PL229': [':pl:', 'Gliwice'],
         'PL22C': [':pl:', 'Tichy'],
         'PL415': [':pl:', 'Poznań']
         
    }

    df = df[df['NUTS ID'].isin(corresp.keys())]
    
    
    # Creates columns with real names, code for datawrapper flags and a combination of both
    df['code'] = df['NUTS ID'].apply(lambda code: corresp[code][0])
    df['city'] = df['NUTS ID'].apply(lambda code: corresp[code][1])
    df['display name'] = df['NUTS ID'].apply(lambda code: corresp[code][0] + " " + corresp[code][1])
    df['year'] = df['Day'].str.slice(0,4)
    
    # Keeps only columns of interest
    df = df[['year', 'code', 'city', 'display name', 'Daily PM 2.5 average (µg/m³)']]

    # Alphabetical order
    df = df.sort_values(by='display name', ascending=False)    
    
    # Pivots so datawrapper reads each year as a column
    df = df.pivot(index=['city', 'code', 'display name'], columns='year').reset_index().droplevel(level=0, axis=1)
    
    # Adds one extra difference column
    df['difference'] = (df['2022'] - df['2018']) / df['2018'] * 100
    
    # Shows
    display(df)
    
    # Saves
    df.to_csv("./datawrapper/sparklines.csv", index=False)

In [ ]:
sparkline_data()